# Quick draw recognition with CNN
In this project I am going to use the Quick draw dataset. Quick draw is one of the Google's A.I. experiments - https://quickdraw.withgoogle.com
I will first train the model. Then convert it to a tensorflow.js model and then I will create a web application with a canvas drawing space that will recognise the drawing using the generated model. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
import glob
import json
import cv2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import random
from tensorflow.keras.optimizers import Adam

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# Any results you write to the current directory are saved as output.

In [ ]:
TRAIN_FILES = glob.glob('../input/shuffle-csvs*/*.csv.gz')
TEST_FILE = pd.read_csv('../input/shuffle-csvs/train_k7.csv.gz', nrows=30000)
# TEST_FILE = pd.read_csv('../input/quickdraw-doodle-recognition/test_simplified.csv', nrows=100)
BASE_SIZE = 256
BATCH_SIZE = 512
NCATS = 340
np.random.seed(seed=1978)

In [ ]:
def draw_image_from_strokes(raw_strokes, size=256, lw=6, augmentation = False, time_color=False):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 0
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        img = cv2.resize(img, (size, size))
    if augmentation:
        if random.random() > 0.5:
            img = np.fliplr(img)
    return img

In [ ]:
def image_generator(size, batchsize, lw=6, augmentation = True, time_color=True):
    while True:
        for filename in TRAIN_FILES:
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 1))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 0] = draw_image_from_strokes(raw_strokes, size=size, lw=lw, augmentation = augmentation,
                                                           time_color=time_color)
                x = x / 255.
                x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y
                
    
def test_image_generator(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(json.loads)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_image_from_strokes(raw_strokes, size=size, lw=lw, time_color=time_color)
    x = x / 255.
    x = preprocess_input(x).astype(np.float32)
    y = keras.utils.to_categorical(df.y, num_classes=NCATS)
    return x, y

In [ ]:
train_datagen = image_generator(128, BATCH_SIZE)

In [ ]:
x, y = next(train_datagen)

In [ ]:
def plot_images(x):
    n = 8
    fig, axs = plt.subplots(nrows=n, ncols=n, sharex=True, sharey=True, figsize=(12, 12))
    for i in range(n**2):
        ax = axs[i // n, i % n]
        (-x[i]+1)/2
        ax.imshow((-x[i, :, :, 0] + 1)/2, cmap=plt.cm.gray)
        ax.axis('off')
    plt.tight_layout()
    fig.savefig('gs.png', dpi=300)
    plt.show();

In [ ]:
plot_images(x)

In [ ]:
x_test, y_test = test_image_generator(TEST_FILE, 128)

In [ ]:
plot_images(x_test)

# THE MODELS

I am creating two models. The first one is custom CNN with just 9 layers.

In [ ]:
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3), padding='same', input_shape=x.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(16, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(16, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(340, activation='softmax'))

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(learning_rate=0.002),
             metrics=[categorical_accuracy, categorical_crossentropy, top_3_accuracy])
print(model.summary())

And the second is MobileNet which is based on a streamlined architecture that uses depthwise separable convolutions to build light weight deep neural networks.

In [ ]:
model_mn = MobileNet(input_shape=(128, 128, 1), alpha=1., weights=None, classes=NCATS)
model_mn.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model_mn.summary())

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.75, patience=3, min_delta=0.001,
                          mode='max', min_lr=1e-5, verbose=1),
    ModelCheckpoint('model.h5', monitor='val_top_3_accuracy', mode='max', save_best_only=True,
                    save_weights_only=True),
]

In [ ]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=BATCH_SIZE, epochs=70, verbose=1,
    validation_data=(x_test, y_test),
    callbacks = callbacks
)
hists.append(hist)